# Linear Regression [35pts (+5 bonus)]

## Introduction
One of the most widespread regression tools is the simple but powerful linear regression. In this notebook, you will engineer the Pittsburgh bus data into numerical features and use them to predict the number of minutes until the bus reaches the bus stop at Forbes and Morewood. 

Notebook restriction: you may not use scikit-learn for this notebook.  

## Q1: Labeling the Dataset [8pts]

You may have noticed that the Pittsburgh bus data has a predictions table with the TrueTime predictions on arrival time, however it does not have the true label: the actual number of minutes until a bus reaches Forbes and Morewood. You will have to generate this yourself. 

Using the `all_trips` function that you implemented in homework 2, you can split the dataframe into separate trips. You will first process each trip into a form more natural for the regression setting. For each trip, you will need to locate the point at which a bus passes the bus stop to get the time at which the bus passes the bus stop. From here, you can calculate the true label for all prior datapoints, and throw out the rest. 

### Importing functions from homework 2

Using the menu in Jupyter, you can import code from your notebook as a Python script using the following steps: 
1. Click File -> Download as -> Python (.py)
2. Save file (time_series.py) in the same directory as this notebook 
3. (optional) Remove all test code (i.e. lines between AUTOLAB_IGNORE macros) from the script for faster loading time
4. Import from the notebook with `from time_series import function_name`

### Specifications

1. To determine when the bus passes Morewood, we will use the Euclidean distance as a metric to determine how close the bus is to the bus stop. 
2. We will assume that the row entry with the smallest Euclidean distance to the bus stop is when the bus reaches the bus stop, and that you should truncate all rows that occur **after** this entry.  In the case where there are multiple entries with the exact same minimal distance, you should just consider the first one that occurs in the trip (so truncate everything after the first occurance of minimal distance). 
3. Assume that the row with the smallest Euclidean distance to the bus stop is also the true time at which the bus passes the bus stop. Using this, create a new column called `eta` that contains for each row, the number of minutes until the bus passes the bus stop (so the last row of every trip will have an `eta` of 0).
4. Make sure your `eta` is numerical and not a python timedelta object. 

In [1]:
import pandas as pd
import numpy as np
import scipy.linalg as la
from collections import Counter

In [2]:
# AUTOLAB_IGNORE_START
from time_series import load_data, split_trips

vdf, _ = load_data('bus_train.db')
all_trips = split_trips(vdf)
# AUTOLAB_IGNORE_STOP

In [3]:
def euclidean(a, b):
    return np.sqrt(np.power(a[0] - b[0], 2) + np.power(a[1] - b[1], 2))

def label_and_truncate(trip, bus_stop_coordinates):
    """ Given a dataframe of a trip following the specification in the previous homework assignment,
        generate the labels and throw away irrelevant rows. 
        
        Args: 
            trip (dataframe): a dataframe from the list outputted by split_trips from homework 2
            stop_coordinates ((float, float)): a pair of floats indicating the (latitude, longitude) 
                                               coordinates of the target bus stop. 
            
        Return:
            (dataframe): a labeled trip that is truncated at Forbes and Morewood and contains a new column 
                         called `eta` which contains the number of minutes until it reaches the bus stop. 
        """
    
    min_dist = float('inf')
    i = 0
    min_i = 0
    min_time = 0
    for index, stop in trip.iterrows():
        coordinate = (stop["lat"], stop["lon"])
        dist = euclidean(bus_stop_coordinates, coordinate)
        i += 1
        if dist < min_dist:
            min_dist = dist
            min_time = index
            min_i = i
    trunc_trip = trip[:min_i]
    trunc_trip = trunc_trip.assign(eta=lambda x: (min_time - x.index).seconds//60)
    return trunc_trip
    
# AUTOLAB_IGNORE_START
morewood_coordinates = (40.444671114203, -79.94356058465502) # (lat, lon)
labeled_trips = [label_and_truncate(trip, morewood_coordinates) for trip in all_trips]
labeled_vdf = pd.concat(labeled_trips).reset_index()
# # We remove datapoints that make no sense (ETA more than 10 hours)
labeled_vdf = labeled_vdf[labeled_vdf["eta"] < 10*60].reset_index(drop=True)
print Counter([len(t) for t in labeled_trips])
print labeled_vdf.head()
# AUTOLAB_IGNORE_STOP

Counter({1: 506, 21: 200, 18: 190, 20: 184, 19: 163, 16: 162, 22: 159, 17: 151, 23: 139, 31: 132, 15: 128, 2: 125, 34: 112, 32: 111, 33: 101, 28: 98, 14: 97, 30: 95, 35: 95, 29: 93, 24: 90, 25: 89, 37: 86, 27: 83, 39: 83, 38: 82, 36: 77, 26: 75, 40: 70, 13: 62, 41: 53, 44: 52, 42: 47, 6: 44, 5: 39, 12: 39, 46: 39, 7: 38, 3: 36, 45: 33, 47: 33, 43: 31, 48: 27, 4: 26, 49: 26, 11: 25, 50: 25, 10: 23, 51: 23, 8: 19, 9: 18, 53: 16, 54: 15, 52: 14, 55: 14, 56: 8, 57: 3, 58: 3, 59: 3, 60: 3, 61: 1, 62: 1, 67: 1})
               tmstmp   vid        lat        lon  hdg   pid   rt  \
0 2016-08-16 11:13:00  5299  40.440145 -79.998813  113  4373  61C   
1 2016-08-16 11:14:00  5299  40.440018 -79.998451  113  4373  61C   
2 2016-08-16 11:15:00  5299  40.439172 -79.996055  107  4373  61C   
3 2016-08-16 11:16:00  5299  40.439011 -79.995085   84  4373  61C   
4 2016-08-16 11:17:00  5299  40.438231 -79.994353  152  4373  61C   

           des  pdist  spd tablockid  tatripid  eta  
0  McKeesport     5

For our implementation, this returns the following output
```python
>>> Counter([len(t) for t in labeled_trips])
Counter({1: 506, 21: 200, 18: 190, 20: 184, 19: 163, 16: 162, 22: 159, 17: 151, 23: 139, 31: 132, 15: 128, 2: 125, 34: 112, 32: 111, 33: 101, 28: 98, 14: 97, 30: 95, 35: 95, 29: 93, 24: 90, 25: 89, 37: 86, 27: 83, 39: 83, 38: 82, 36: 77, 26: 75, 40: 70, 13: 62, 41: 53, 44: 52, 42: 47, 6: 44, 5: 39, 12: 39, 46: 39, 7: 38, 3: 36, 45: 33, 47: 33, 43: 31, 48: 27, 4: 26, 49: 26, 11: 25, 50: 25, 10: 23, 51: 23, 8: 19, 9: 18, 53: 16, 54: 15, 52: 14, 55: 14, 56: 8, 57: 3, 58: 3, 59: 3, 60: 3, 61: 1, 62: 1, 67: 1}) 
>>> labeled_vdf.head()
               tmstmp   vid        lat        lon  hdg   pid   rt        des  \
0 2016-08-11 10:56:00  5549  40.439504 -79.996981  114  4521  61A  Swissvale   
1 2016-08-11 10:57:00  5549  40.439504 -79.996981  114  4521  61A  Swissvale   
2 2016-08-11 10:58:00  5549  40.438842 -79.994733  124  4521  61A  Swissvale   
3 2016-08-11 10:59:00  5549  40.437938 -79.991213   94  4521  61A  Swissvale   
4 2016-08-11 10:59:00  5549  40.437938 -79.991213   94  4521  61A  Swissvale   

   pdist  spd tablockid  tatripid  eta  
0   1106    0  061A-164      6691   16  
1   1106    0  061A-164      6691   15  
2   1778    8  061A-164      6691   14  
3   2934    7  061A-164      6691   13  
4   2934    7  061A-164      6691   13 
```

## Q2: Generating Basic Features [8pts]
In order to perform linear regression, we need to have numerical features. However, not everything in the bus database is a number, and not all of the numbers even make sense as numerical features. If you use the data as is, it is highly unlikely that you'll achieve anything meaningful.

Consequently, you will perform some basic feature engineering. Feature engineering is extracting "features" or statistics from your data, and hopefully improve the performance if your learning algorithm (in this case, linear regression). Good features can often make up for poor model selection and improve your overall predictive ability on unseen data. In essence, you want to turn your data into something your algorithm understands. 

### Specifications
1. The input to your function will be a concatenation of the trip dataframes generated in Q1 with the index dropped (so same structure as the original dataframe, but with an extra column and less rows). 
2. Linear models typically have a constant bias term. We will encode this as a column of 1s in the dataframe. Call this column 'bias'. 
2. We will keep the following columns as is, since they are already numerical:  pdist, spd, lat, lon, and eta 
3. Time is a cyclic variable. To encode this as a numerical feature, we can use a sine/cosine transformation. Suppose we have a feature of value f that ranges from 0 to N. Then, the sine and cosine transformation would be $\sin\left(2\pi \frac{f}{N}\right)$ and $\cos\left(2\pi \frac{f}{N}\right)$. For example, the sine transformation of 6 hours would be $\sin\left(2\pi \frac{6}{24}\right)$, since there are 24 hours in a cycle. You should create sine/cosine features for the following:
    * day of week (cycles every week, 0=Monday)
    * hour of day (cycles every 24 hours, 0=midnight)
    * time of day represented by total number of minutes elapsed in the day (cycles every 60*24 minutes, 0=midnight).
4. Heading is also a cyclic variable, as it is the ordinal direction in degrees (so cycles every 360 degrees). 
4. Buses run on different schedules on the weekday as opposed to the weekend. Create a binary indicator feature `weekday` that is 1 if the day is a weekday, and 0 otherwise. 
5. Route and destination are both categorical variables. We can encode these as indicator vectors, where each column represents a possible category and a 1 in the column indicates that the row belongs to that category. This is also known as a one hot encoding. Make a set of indicator features for the route, and another set of indicator features for the destination. 
6. The names of your indicator columns for your categorical variables should be exactly the value of the categorical variable. The pandas function `pd.DataFrame.get_dummies` will be useful. 

In [4]:
import math

def get_sin_of_day(x):
    return x

def get_busday(x):  
    return 1 if np.is_busday(x) else 1

def get_sin_hdg(x):
    return np.sin(2.0 * math.pi * x / 360.0)

def get_cos_hdg(x):
    return np.cos(2.0 * math.pi * x / 360.0)

def get_sin_day(x):
    return np.sin(2.0 * math.pi * x.weekday() / 7.0)

def get_cos_day(x):
    return np.cos(2.0 * math.pi * x.weekday() / 7.0)

def get_sin_hour(x):
    return np.sin(2.0 * math.pi * x.hour / 24.0)

def get_cos_hour(x):
    return np.cos(2.0 * math.pi * x.hour / 24.0)

def get_sin_time(x):
    return np.sin(2.0 * math.pi * (x.hour * 60.0 + x.minute) / (60.0 * 24.0))

def get_cos_time(x):
    return np.cos(2.0 * math.pi * (x.hour * 60.0 + x.minute) / (60.0 * 24.0))

def create_features(vdf):
    """ Given a dataframe of labeled and truncated bus data, generate features for linear regression. 
    
        Args:
            df (dataframe) : dataframe of bus data with the eta column and truncated rows
        Return: 
            (dataframe) : dataframe of features for each example
        """
#     u'sin_hdg',           u'cos_hdg',   u'sin_day_of_week',
#          u'cos_day_of_week',   u'sin_hour_of_day',   u'cos_hour_of_day',
#          u'sin_time_of_day',   u'cos_time_of_day',           u'weekday',
    vdf = vdf.assign(bias=lambda x: 1) 
    vdf = vdf[["bias", "pdist", "spd", "lat", "lon", "eta", "hdg", "tmstmp", "des", "rt"]]
    vdf = vdf.assign(sin_hdg=lambda x: x["hdg"].apply(get_sin_hdg))
    vdf = vdf.assign(cos_hdg=lambda x: x["hdg"].apply(get_cos_hdg))
    vdf = vdf.assign(sin_day_of_week=lambda x: x["tmstmp"].apply(get_sin_day))
    vdf = vdf.assign(cos_day_of_week=lambda x: x["tmstmp"].apply(get_cos_day))
    vdf = vdf.assign(sin_hour_of_day=lambda x: x["tmstmp"].apply(get_sin_hour))
    vdf = vdf.assign(cos_hour_of_day=lambda x: x["tmstmp"].apply(get_cos_hour))
    vdf = vdf.assign(sin_time_of_day=lambda x: x["tmstmp"].apply(get_sin_time))
    vdf = vdf.assign(cos_time_of_day=lambda x: x["tmstmp"].apply(get_cos_time))
    vdf = vdf.assign(weekday=lambda x: x["tmstmp"].apply(get_busday))
    vdf = pd.get_dummies(vdf, prefix=['', ''], prefix_sep='')
    return vdf.drop("tmstmp", axis=1).drop("hdg", axis=1)

# AUTOLAB_IGNORE_START
vdf_features = create_features(labeled_vdf)
# AUTOLAB_IGNORE_STOP

In [5]:
# AUTOLAB_IGNORE_START
with pd.option_context('display.max_columns', 26):
    print vdf_features.columns
    print vdf_features.head()
# AUTOLAB_IGNORE_STOP

Index([             u'bias',             u'pdist',               u'spd',
                     u'lat',               u'lon',               u'eta',
                 u'sin_hdg',           u'cos_hdg',   u'sin_day_of_week',
         u'cos_day_of_week',   u'sin_hour_of_day',   u'cos_hour_of_day',
         u'sin_time_of_day',   u'cos_time_of_day',           u'weekday',
               u'Braddock ',          u'Downtown',   u'Greenfield Only',
             u'McKeesport ', u'Murray-Waterfront',         u'Swissvale',
                     u'61A',               u'61B',               u'61C',
                     u'61D'],
      dtype='object')
   bias  pdist  spd        lat        lon  eta   sin_hdg   cos_hdg  \
0     1    550    0  40.440145 -79.998813   19  0.920505 -0.390731   
1     1    660   20  40.440018 -79.998451   18  0.920505 -0.390731   
2     1   1366   18  40.439172 -79.996055   17  0.956305 -0.292372   
3     1   1665   18  40.439011 -79.995085   16  0.994522  0.104528   
4     1   2028

Our implementation has the following output. Verify that your code has the following columns (order doesn't matter): 
```python
>>> vdf_features.columns
Index([             u'bias',             u'pdist',               u'spd',
                     u'lat',               u'lon',               u'eta',
                 u'sin_hdg',           u'cos_hdg',   u'sin_day_of_week',
         u'cos_day_of_week',   u'sin_hour_of_day',   u'cos_hour_of_day',
         u'sin_time_of_day',   u'cos_time_of_day',           u'weekday',
               u'Braddock ',          u'Downtown',   u'Greenfield Only',
             u'McKeesport ', u'Murray-Waterfront',         u'Swissvale',
                     u'61A',               u'61B',               u'61C',
                     u'61D'],
      dtype='object')
   bias  pdist  spd        lat        lon  eta   sin_hdg   cos_hdg  \
0   1.0   1106    0  40.439504 -79.996981   16  0.913545 -0.406737   
1   1.0   1106    0  40.439504 -79.996981   15  0.913545 -0.406737   
2   1.0   1778    8  40.438842 -79.994733   14  0.829038 -0.559193   
3   1.0   2934    7  40.437938 -79.991213   13  0.997564 -0.069756   
4   1.0   2934    7  40.437938 -79.991213   13  0.997564 -0.069756   

   sin_day_of_week  cos_day_of_week ...   Braddock   Downtown  \
0         0.433884        -0.900969 ...         0.0       0.0   
1         0.433884        -0.900969 ...         0.0       0.0   
2         0.433884        -0.900969 ...         0.0       0.0   
3         0.433884        -0.900969 ...         0.0       0.0   
4         0.433884        -0.900969 ...         0.0       0.0   

   Greenfield Only  McKeesport   Murray-Waterfront  Swissvale  61A  61B  61C  \
0              0.0          0.0                0.0        1.0  1.0  0.0  0.0   
1              0.0          0.0                0.0        1.0  1.0  0.0  0.0   
2              0.0          0.0                0.0        1.0  1.0  0.0  0.0   
3              0.0          0.0                0.0        1.0  1.0  0.0  0.0   
4              0.0          0.0                0.0        1.0  1.0  0.0  0.0   

   61D  
0  0.0  
1  0.0  
2  0.0  
3  0.0  
4  0.0  

[5 rows x 25 columns]
```

## Q3 Linear Regression using Ordinary Least Squares [10 + 4pts]
Now you will finally implement a linear regression. As a reminder, linear regression models the data as

$$\mathbf y = \mathbf X\mathbf \beta + \mathbf \epsilon$$

where $\mathbf y$ is a vector of outputs, $\mathbf X$ is also known as the design matrix, $\mathbf \beta$ is a vector of parameters, and $\mathbf \epsilon$ is noise. We will be estimating $\mathbf \beta$ using Ordinary Least Squares, and we recommending following the matrix notation for this problem (https://en.wikipedia.org/wiki/Ordinary_least_squares). 

### Specification
1. We use the numpy term array-like to refer to array like types that numpy can operate on (like Pandas DataFrames). 
1. Regress the output (eta) on all other features
2. Return the predicted output for the inputs in X_test
3. Calculating the inverse $(X^TX)^{-1}$ is unstable and prone to numerical inaccuracies. Furthermore, the assumptions of Ordinary Least Squares require it to be positive definite and invertible, which may not be true if you have redundant features. Thus, you should instead use $(X^TX + \lambda*I)^{-1}$ for identity matrix $I$ and $\lambda = 10^{-4}$, which for now acts as a numerical "hack" to ensure this is always invertible. Furthermore, instead of computing the direct inverse, you should utilize the Cholesky decomposition which is much more stable when solving linear systems. 

In [12]:
class LR_model():
    """ Perform linear regression and predict the output on unseen examples. 
        Attributes: 
            beta (array_like) : vector containing parameters for the features """
    
    def __init__(self, X, y):
        """ Initialize the linear regression model by computing the estimate of the weights parameter
            Args: 
                X (array-like) : feature matrix of training data where each row corresponds to an example
                y (array like) : vector of training data outputs 
            """       
        lam = 0.0001
        unit_matrix = np.eye(X.shape[1])
#         L = np.linalg.cholesky()
#         L_inv = np.linalg.inv(L)
#         inv = L_inv.transpose().dot(L_inv)
#         self.beta = inv.dot(X.transpose()).dot(y)
        self.beta = np.linalg.solve(X.transpose().dot(X) + lam * unit_matrix, X.transpose().dot(y))
        
    def predict(self, X_p): 
        """ Predict the output of X_p using this linear model. 
            Args: 
                X_p (array_like) feature matrix of predictive data where each row corresponds to an example
            Return: 
                (array_like) vector of predicted outputs for the X_p
            """
        return X_p.dot(self.beta)


We have provided some validation data for you, which is another scrape of the Pittsburgh bus data (but for a different time span). You will need to do the same processing to generate labels and features to your validation dataset. Calculate the mean squared error of the output of your linear regression on both this dataset and the original training dataset. 

How does it perform? One simple baseline is to make sure that it at least predicts as well as predicting the mean of what you have seen so far. Does it do better than predicting the mean? Compare the mean squared error of a predictor that predicts the mean vs your linear classifier. 

### Specifications
1. Build your linear model using only the training data
2. Compute the mean squared error of the predictions on both the training and validation data. 
3. Compute the mean squared error of predicting the mean of the **training outputs** for all inputs. 
4. You will need to process the validation dataset in the same way you processed the training dataset.
5. You will need to split your features from your output (eta) prior to calling compute_mse

In [27]:
def mse(x, y):
    return ((x - y)**2).mean()

# Calculate mean squared error on both the training and validation set
def compute_mse(LR, X, y, X_v, y_v):
    """ Given a linear regression model, calculate the mean squared error for the 
        training dataset, the validation dataset, and for a mean prediction
        Args:
            LR (LR_model) : Linear model
            X (array-like) : feature matrix of training data where each row corresponds to an example
            y (array like) : vector of training data outputs 
            X_v (array-like) : feature matrix of validation data where each row corresponds to an example
            y_v (array like) : vector of validation data outputs 
        Return: 
            (train_mse, train_mean_mse, 
             valid_mse, valid_mean_mse) : a 4-tuple of mean squared errors
                                             1. MSE of linear regression on the training set
                                             2. MSE of predicting the mean on the training set
                                             3. MSE of linear regression on the validation set
                                             4. MSE of predicting the mean on the validation set
                         
            
    """
    y_pred = LR.predict(X)
    y_v_pred = LR.predict(X_v)
    train_mse = mse(y_pred, y)
    valid_mse = mse(y_v_pred, y_v)
    return train_mse, mse(y, y.mean()), valid_mse, mse(y_v, y.mean())



In [21]:
# AUTOLAB_IGNORE_START
# First you should replicate the same processing pipeline as we did to the training set
vdf_valid, pdf_valid = load_data('bus_valid.db')
morewood_coordinates = (40.444671114203, -79.94356058465502) # (lat, lon)
all_trips_valid = split_trips(vdf_valid)
labeled_trips_valid = [label_and_truncate(trip, morewood_coordinates) for trip in all_trips_valid]
labeled_vdf_valid = pd.concat(labeled_trips_valid).reset_index()
labeled_vdf_valid = labeled_vdf_valid[labeled_vdf_valid["eta"] < 10*60].reset_index(drop=True)
vdf_features_valid = create_features(labeled_vdf_valid)

# Separate the features from the output and pass it into your linear regression model.
y_df = vdf_features["eta"]
X_df = vdf_features.drop("eta", axis=1)
y_valid_df = vdf_features_valid["eta"]
X_valid_df = vdf_features_valid.drop("eta", axis=1)

# LR = LR_model(X_df, y_df)
# print compute_mse(LR, 
#                   X_df, 
#                   y_df, 
#                   X_valid_df, 
#                   y_valid_df)

# AUTOLAB_IGNORE_STOP

In [22]:
# AUTOLAB_IGNORE_START
A = np.array([[1, 3, 5], [2, 5, 1], [2, 3, 8]])
y = np.array([10, 8, 3])
a = LR_model(A, y)
print a.beta
# AUTOLAB_IGNORE_STOP

[-9.27653474  5.15872708  0.75973591]


In [29]:
# AUTOLAB_IGNORE_START

LR = LR_model(X_df, y_df)
print compute_mse(LR, 
                  X_df, 
                  y_df, 
                  X_valid_df, 
                  y_valid_df)

# AUTOLAB_IGNORE_STOP

(39.07207047204576, 130.58048316520953, 50.21316911531619, 151.55706838013853)


As a quick check, our training data MSE is approximately 38.99. 

## Q4 TrueTime Predictions [5pts]
How do you fare against the Pittsburgh Truetime predictions? In this last problem, you will match predictions to their corresponding vehicles to build a dataset that is labeled by TrueTime. Remember that we only evaluate performance on the validation set (never the training set). How did you do?

### Specification
1. You should use the pd.DataFrame.merge function to combine your vehicle dataframe and predictions dataframe into a single dataframe. You should drop any rows that have no predictions (see the how parameter). (http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html)
2. You can compute the TrueTime ETA by taking their predicted arrival time and subtracting the timestamp, and converting that into an integer representing the number of minutes. 
3. Compute the mean squared error for linear regression only on the rows that have predictions (so only the rows that remain after the merge). 

In [46]:
def get_peta(row):
    return row["prdtm"] - row["tmstmp"]

def compare_truetime(LR, labeled_vdf, pdf):
    """ Compute the mse of the truetime predictions and the linear regression mse on entries that have predictions.
        Args:
            LR (LR_model) : an already trained linear model
            labeled_vdf (pd.DataFrame): a dataframe of the truncated and labeled bus data (same as the input to create_features)
            pdf (pd.DataFrame): a dataframe of TrueTime predictions
        Return: 
            (tt_mse, lr_mse): a tuple of the TrueTime MSE, and the linear regression MSE
        """
    merged = labeled_vdf.merge(pdf)
    merged_features = create_features(merged)
    X_test_df = merged_features.drop("eta", axis=1)
    y_pred_test = LR.predict(X_test_df)
       
    merged['p_eta'] = merged.apply (lambda row: get_peta(row), axis=1)
    merged = merged.assign(p_eta=lambda x: x["p_eta"].apply(lambda y: y.seconds//60))
    lr_mse = mse(y_pred_test, merged["eta"])   
    tt_mse = mse(merged["p_eta"], merged["eta"])
    return (tt_mse, lr_mse)
    
# AUTOLAB_IGNORE_START
print compare_truetime(LR, labeled_vdf_valid, pdf_valid)
# AUTOLAB_IGNORE_STOP

   bias  pdist  spd        lat        lon  eta   sin_hdg   cos_hdg  \
0     1   7126   20  40.408379 -79.867104   28 -0.882948 -0.469472   
1     1   7789   33  40.406821 -79.866265   27  0.224951 -0.974370   
2     1   8598   26  40.404987 -79.867567   26 -0.731354 -0.681998   
3     1   9286   16  40.403971 -79.869544   25 -0.798636  0.601815   
4     1  10694   38  40.406361 -79.873652   24 -0.819152  0.573576   

   sin_day_of_week  cos_day_of_week ...   Braddock   Downtown  \
0         0.781831          0.62349 ...         0.0       1.0   
1         0.781831          0.62349 ...         0.0       1.0   
2         0.781831          0.62349 ...         0.0       1.0   
3         0.781831          0.62349 ...         0.0       1.0   
4         0.781831          0.62349 ...         0.0       1.0   

   Greenfield Only  McKeesport   Murray-Waterfront  Swissvale  61A  61B  61C  \
0              0.0          0.0                0.0        0.0  0.0  1.0  0.0   
1              0.0          

As a sanity check, your linear regression MSE should be approximately 50.20. 

## Q5 Feature Engineering contest (bonus)

You may be wondering "why did we pick the above features?" Some of the above features may be entirely useless, or you may have ideas on how to construct better features. Sometimes, choosing good features can be the entirety of a data science problem. 

In this question, you are given complete freedom to choose what and how many features you want to generate. Upon submission to Autolab, we will run linear regression on your generated features and maintain a scoreboard of best regression accuracy (measured by mean squared error). 

The top scoring students will receive a bonus of 5 points. 

### Tips:
* Test your features locally by building your model using the training data, and predicting on the validation data. Compute the mean squared error on the **validation dataset** as a metric for how well your features generalize. This helps avoid overfitting to the training dataset, and you'll have faster turnaround time than resubmitting to autolab. 
* The linear regression model will be trained on your chosen features of the same training examples we provide in this notebook. 
* We test your regression on a different dataset from the training and validation set that we provide for you, so the MSE you get locally may not match how your features work on the Autolab dataset. 
* We will solve the linear regression using Ordinary Least Squares with regularization $\lambda=10^{-4}$ and a Cholesky factorization, exactly as done earlier in this notebook. 
* Note that the argument contains **UNlabeled** data: you cannot build features off the output labels (there is no ETA column). This is in contrast to before, where we kept everything inside the same dataframe for convenience. You can produce the sample input by removing the "eta" column, which we provide code for below. 
* Make sure your features are all numeric. Try everything!

In [37]:
def get_trip_char(row):
    return row["rt"] + "_" + row["des"]

def contest_features(vdf, vdf_train):
    """ Given a dataframe of UNlabeled and truncated bus data, generate ANY features you'd like for linear regression. 
        Args:
            vdf (dataframe) : dataframe of bus data with truncated rows but unlabeled (no eta column )
                              for which you should produce features
            vdf_train (dataframe) : dataframe of training bus data, truncated and labeled 
        Return: 
            (dataframe) : dataframe of features for each example in vdf
        """
    # create your own engineered features
    vdf = vdf.assign(bias=lambda x: 1)
    vdf['trip'] = vdf.apply (lambda row: get_trip_char(row), axis=1)
    vdf = vdf[["bias", "pdist", "lat", "lon", "spd", "hdg", "tmstmp", "trip"]]    
    vdf = vdf.assign(sin_hdg=lambda x: x["hdg"].apply(get_sin_hdg))
    vdf = vdf.assign(sin_day_of_week=lambda x: x["tmstmp"].apply(get_sin_day))
    vdf = vdf.assign(sin_time_of_day=lambda x: x["tmstmp"].apply(get_sin_time))
    vdf = vdf.assign(weekday=lambda x: x["tmstmp"].apply(get_busday))
    vdf = pd.get_dummies(vdf, prefix=[''], prefix_sep='')
    return vdf.drop("tmstmp", axis=1).drop("hdg", axis=1)
    
# AUTOLAB_IGNORE_START
contest_cols = list(labeled_vdf.columns)
contest_cols.remove("eta")
contest_features(labeled_vdf_valid[contest_cols], labeled_vdf).head()
# AUTOLAB_IGNORE_STOP

,bias,pdist,lat,lon,spd,sin_hdg,sin_day_of_week,sin_time_of_day,weekday,61A_Downtown,61A_Swissvale,61B_Braddock,61B_Downtown,61C_Downtown,61C_McKeesport,61D_Downtown,61D_Greenfield Only,61D_Murray-Waterfront
0,1,112,40.402080,-79.868343,0,0.75471,0.781831,0.358368,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,60,40.402416,-79.868240,0,0.75471,0.781831,0.354291,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1,60,40.402416,-79.868240,0,0.75471,0.781831,0.354291,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1,60,40.402416,-79.868240,0,0.75471,0.781831,0.346117,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1,60,40.402416,-79.868240,0,0.75471,0.781831,0.342020,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
